In [ ]:
from preprocess import *
from train import *
from load import *
import time
import random
from torch import optim
import torch.utils.data as data
from tqdm import tqdm
from models import *

In [ ]:
# Update to change the dataset
dname = 'Foursquare'

In [ ]:
# # Generate the cleaned npy files from the raw data
# preprocess(dname)

In [ ]:
# Generate the initial embeddings as pickle file from above npy files
process_traj(dname)

In [ ]:
# Train the model and get the results
###hyper_parameters
# part: 5, 10, 20, 40, 60, 80, 100
# emb_dim: 10, 20, 40, 64, 128, 256, 512, 1024
# num_neg: 1, 10, 15, 20, 30, 40, 50
# fixed: part = 100, emb_dim = 256, num_neg = 10, epochs = 30, dropout = 0, lr = 3e-3, load = False

part = 100
emb_dim = 512
dropout = 0
load = False
num_neg = 10
lr = 3e-3
epochs = 100

file = open('./data/' + dname + '_data.pkl', 'rb') 
file_data = joblib.load(file)
# tensor(NUM, M, 3), np(NUM, M, M, 2), np(L, L), np(NUM, M, M), tensor(NUM, M), np(NUM)
[trajs, mat1, mat2s, mat2t, labels, lens, u_max, l_max] = file_data
mat1, mat2s, mat2t, lens = torch.FloatTensor(mat1), torch.FloatTensor(mat2s).to(device), torch.FloatTensor(mat2t), torch.LongTensor(lens)


trajs, mat1, mat2t, labels, lens = trajs[:part], mat1[:part], mat2t[:part], labels[:part], lens[:part]
ex = mat1[:, :, :, 0].max(), mat1[:, :, :, 0].min(), mat1[:, :, :, 1].max(), mat1[:, :, :, 1].min()
stan = Model(t_dim=hours+1, l_dim=l_max+1, u_dim=u_max+1, embed_dim=emb_dim, ex=ex, dropout=dropout)

# for name in stan.state_dict():
#     print(name)

num_params = 0
for param in stan.parameters():
    num_params += param.numel()
print('num of params', num_params)

if load:
    checkpoint = torch.load('best_stan_win_' + dname + '.pth')
    stan.load_state_dict(checkpoint['state_dict'])
    start = time.time() - checkpoint['time']
    records = checkpoint['records']
else:
    records = {'epoch': [], 'recall_valid': [], 'recall_test': [], 'ndcg_valid': [], 'ndcg_test': []}
    start = time.time()

trainer = Trainer(stan, records, load, trajs, mat1, mat2t, labels, lens, mat2s, num_neg, lr, epochs)
trainer.train(part, start, dname, emb_dim)